# Job Scheduling
![footer_logo_new](images/logo_new.png)

In the cluster mode, each Spark application runs an independent set of Executor processes. 

Cluster Manager Provides facilities for scheduling accross multiple Spark Applications.

### Jobs, Stages and Tasks

Each job might consist of 1-N Stages, each stage might consist of 1-N Tasks.

![jobs_stages_tasks](images/jobs_stages_tasks.png)

## Scheduling

The process of scheduling is providing access to the physical resources in the cluster.

#### Slot
In Spark there is concept of a Slot - minimum amount of resources needed to run a Task. Usually a Executor CPU == Slot.

![tasks_executors_slots](images/tasks_executors_slots.png)

## Static Resources Partitioning (Allocation)

In most cluster implementations there is an option to fix the maximum amount of resources a Spark Applicaiton will use. It hold to this resources untill completion.

Depending on the cluster implementation, resource allocation can be configured as:

- Standalone Mode: by default applications run in FIFO order. Each application will try to use all available resources. You can limit the amount of resources an application might use.
- Mesos: configure `spark.mesos.coarse=true`, `spark.cores.max` and `spark.executor.memory`
- YARN: `spark.executor.instances`, `spark.executor.memory`, `spark.executor.cores`

## Dynamic Resources Allocation

In Mesos, YARN and Kubernetes cluster managers it is possible to configure dynamic resources allocation occupied by an application.

Dynamic Allocation means that depending on the current state of the Tasks Queue, an application either will get more resources or it will give the resources back.

#### Request Policy

Spark requests resources in rounds with exponential growth based on the number of tasks in the previous request round.

The actual request is triggered when there have been pending tasks for `spark.dynamicAllocation.schedulerBacklogTimeout` seconds, and then triggered again every `spark.dynamicAllocation.sustainedSchedulerBacklogTimeout` seconds thereafter if the queue of pending tasks persists. 

Additionally, the number of executors requested in each round increases exponentially from the previous round. For instance, an application will add 1 executor in the first round, and then 2, 4, 8 and so on executors in the subsequent rounds.

#### Remove Policy

A Spark application removes an executor when it has been idle for more than `spark.dynamicAllocation.executorIdleTimeout` seconds.

## Scheduling Within a Spark Application

Inside a given Spark Application (SparkContext instance), multiple parallel Jobs can run simultaneously if they were submitted from separate threads. For example network serving application.

#### FIFO Scheduler
First-In-First-Out schedules jobs in a sequential manner. First job submitted will get access to a set of resources and use them until completion. Then next submitted job will get access to resources.  
`spark.scheduler.mode=FIFO`

#### FAIR Scheduler
Under fair sharing, Spark assigns tasks between jobs in a “round robin” fashion, so that all jobs get a roughly equal share of cluster resources. Short jobs submitted while a long job is running can start receiving resources right away and still get good response times, without waiting for the long job to finish.  
`spark.scheduler.mode=FAIR`

#### Pools
The fair scheduler also supports grouping jobs into pools, and setting different scheduling options (e.g. weight) for each pool.  
`sc.setLocalProperty("spark.scheduler.pool", "pool1")`

By default, each pool gets an equal share of the cluster (also equal in share to each job in the default pool), but inside each pool, jobs run in FIFO order. 

##### Configuration
```
<?xml version="1.0"?>
<allocations>
  <pool name="production">
    <schedulingMode>FAIR</schedulingMode>
    <weight>1</weight>
    <minShare>2</minShare>
  </pool>
  <pool name="test">
    <schedulingMode>FIFO</schedulingMode>
    <weight>2</weight>
    <minShare>3</minShare>
  </pool>
</allocations>
```

- **schedulingMode**: This can be FIFO or FAIR, to control whether jobs within the pool queue up behind each other (the default) or share the pool’s resources fairly.
- **weight**: This controls the pool’s share of the cluster relative to other pools. By default, all pools have a weight of 1. If you give a specific pool a weight of 2, for example, it will get 2x more resources as other active pools. Setting a high weight such as 1000 also makes it possible to implement priority between pools—in essence, the weight-1000 pool will always get to launch tasks first whenever it has jobs active.
- **minShare**: Apart from an overall weight, each pool can be given a minimum shares (as a number of CPU cores) that the administrator would like it to have. The fair scheduler always attempts to meet all active pools’ minimum shares before redistributing extra resources according to the weights. The minShare property can, therefore, be another way to ensure that a pool can always get up to a certain number of resources (e.g. 10 cores) quickly without giving it a high priority for the rest of the cluster. By default, each pool’s minShare is 0.